<a href="https://colab.research.google.com/github/1moi6/pyfuzzy-toolbox/blob/main/notebooks_colab/Aula_3/03_wang_mendel_iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌸 Classificação de Flores Iris com Wang-Mendel

**Aula 3 - Minicurso de Sistemas de Inferência Fuzzy**

---

## 📋 Objetivo

Neste notebook, vamos aplicar o **Método de Wang-Mendel** para classificar flores Iris usando apenas **2 variáveis**:
- **Petal Length** (comprimento da pétala)
- **Petal Width** (largura da pétala)

### Por que Iris?
- Dataset clássico de Machine Learning (Fisher, 1936)
- 3 espécies: *Setosa*, *Versicolor*, *Virginica*
- 150 amostras (50 de cada espécie)
- **Vantagem do Wang-Mendel**: Regras interpretáveis!

---

## 📚 Referências
- Wang, L. X., & Mendel, J. M. (1992). "Generating fuzzy rules by learning from examples."
- Fisher, R. A. (1936). "The use of multiple measurements in taxonomic problems."

---

## 🔧 Instalação

In [ ]:
!pip install pyfuzzy-toolbox[ml] scikit-learn -q

print('✅ pyfuzzy-toolbox e scikit-learn instalados com sucesso!')

## 📦 Importações

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import fuzzy_systems as fs
from fuzzy_systems.learning import WangMendelClassification
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

# Configuração
plt.rcParams['figure.figsize'] = (12, 6)
np.set_printoptions(precision=3, suppress=True)
np.random.seed(42)

print('✅ Bibliotecas importadas!')
print(f'   pyfuzzy-toolbox: {fs.__version__}')

## 📊 Passo 1: Carregar e Explorar Dataset Iris

In [ ]:
# Carregar dataset Iris
iris = load_iris()

# Usar apenas features 2 e 3 (Petal Length e Petal Width)
X = iris.data[:, 2:4]
y = iris.target

feature_names = ['Petal Length (cm)', 'Petal Width (cm)']
class_names = ['setosa', 'versicolor', 'virginica']

print('📊 Dataset Iris (2 variáveis)')
print(f'   Shape: {X.shape}')
print(f'   Features: {feature_names}')
print(f'   Classes: {class_names}')
print()

# Estatísticas
print('📈 Estatísticas:')
for i, name in enumerate(feature_names):
    print(f'   {name:20s}: [{X[:, i].min():.2f}, {X[:, i].max():.2f}]')
print()

# Distribuição de classes
print('🌸 Distribuição de classes:')
for i, name in enumerate(class_names):
    count = np.sum(y == i)
    print(f'   {name:12s}: {count} amostras')

## 📈 Visualização dos Dados

In [ ]:
# Scatter plot colorido por classe
plt.figure(figsize=(10, 7))

colors = ['red', 'green', 'blue']
markers = ['o', 's', '^']

for i, (name, color, marker) in enumerate(zip(class_names, colors, markers)):
    idx = y == i
    plt.scatter(X[idx, 0], X[idx, 1], 
                c=color, marker=marker, s=100, 
                label=name, alpha=0.7, edgecolors='black')

plt.xlabel('Petal Length (cm)', fontsize=12)
plt.ylabel('Petal Width (cm)', fontsize=12)
plt.title('Dataset Iris - Visualização 2D', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print('✅ As classes são visualmente separáveis!')

## 📦 Passo 2: Dividir em Treino e Teste

In [ ]:
# Dividir dataset: 70% treino, 30% teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print('📦 Divisão dos dados:')
print(f'   Treino: {len(X_train)} amostras')
print(f'   Teste:  {len(X_test)} amostras')
print()
print('✅ Dados prontos para Wang-Mendel!')

## 🧠 Passo 3: Criar e Treinar Modelo Wang-Mendel

Vamos criar um modelo com:
- **5 partições para cada entrada** (petal_length e petal_width)
- **Classificação** em 3 classes (setosa, versicolor, virginica)

In [ ]:
# Criar modelo Wang-Mendel para classificação
wm = WangMendel(
    n_partitions=5,
    margin=0.15,
    input_names=['petal_length', 'petal_width'],
    output_name='species',
    classification=True,
    n_classes=3
)

print('✅ Modelo Wang-Mendel criado para classificação!')
print('   • 5 partições para cada entrada')
print('   • 3 classes: setosa, versicolor, virginica')
print('   • Margem: 15%\n')

# Treinar (executa os 5 passos automaticamente)
wm.fit(X_train, y_train)

print('\n✅ Treinamento completo!')
print(f'   • Regras geradas: {wm.n_rules}')

## 📊 Passo 4: Visualizar Partições Fuzzy

In [ ]:
# Visualizar partições das entradas
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Partições de petal_length
x1_universe = np.linspace(wm.fis.inputs['petal_length'].universe[0], 
                          wm.fis.inputs['petal_length'].universe[1], 300)
for term_name, term in wm.fis.inputs['petal_length'].terms.items():
    mu = term.membership(x1_universe)
    axes[0].plot(x1_universe, mu, linewidth=2, label=term_name, alpha=0.7)
    axes[0].fill_between(x1_universe, 0, mu, alpha=0.2)

axes[0].set_title('Partições de Petal Length (5 termos)', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Petal Length (cm)', fontsize=12)
axes[0].set_ylabel('Grau de Pertinência', fontsize=12)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([-0.05, 1.05])

# Partições de petal_width
x2_universe = np.linspace(wm.fis.inputs['petal_width'].universe[0], 
                          wm.fis.inputs['petal_width'].universe[1], 300)
for term_name, term in wm.fis.inputs['petal_width'].terms.items():
    mu = term.membership(x2_universe)
    axes[1].plot(x2_universe, mu, linewidth=2, label=term_name, alpha=0.7)
    axes[1].fill_between(x2_universe, 0, mu, alpha=0.2)

axes[1].set_title('Partições de Petal Width (5 termos)', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Petal Width (cm)', fontsize=12)
axes[1].set_ylabel('Grau de Pertinência', fontsize=12)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([-0.05, 1.05])

plt.tight_layout()
plt.show()

print('✅ Partições visualizadas!')

## 📜 Passo 5: Inspecionar Regras Geradas

In [ ]:
print('='*70)
print('📜 REGRAS GERADAS AUTOMATICAMENTE')
print('='*70)
print(f'\nTotal de regras: {len(wm.fis.rules)}\n')

print('Exemplos de regras (mostrando primeiras 10):\n')
for i, rule in enumerate(wm.fis.rules[:10], 1):
    # Extrair antecedentes e consequentes
    ant_str = ' E '.join([f'{var} é {term}' for var, term in rule.antecedents.items()])
    cons_str = ', '.join([f'{var} é {term}' for var, term in rule.consequent.items()])
    print(f'   {i:2d}. SE {ant_str}')
    print(f'       ENTÃO {cons_str}')

if len(wm.fis.rules) > 10:
    print(f'\n   ... (e mais {len(wm.fis.rules) - 10} regras)')

print('\n' + '='*70)
print('💡 Regras capturam padrões para classificar as 3 espécies!')
print('='*70)

## 🧪 Passo 6: Fazer Predições e Avaliar

In [ ]:
# Fazer predições
y_pred_fuzzy = wm.predict(X_test)

# Calcular acurácia
acc_fuzzy = accuracy_score(y_test, y_pred_fuzzy)

print('='*70)
print('📊 RESULTADOS DO WANG-MENDEL')
print('='*70)
print(f'\n✅ Acurácia: {acc_fuzzy*100:.2f}%\n')

# Relatório de classificação
print('📋 Relatório de Classificação:')
print(classification_report(y_test, y_pred_fuzzy, target_names=class_names))
print('='*70)

## 📈 Matriz de Confusão

In [ ]:
# Calcular matriz de confusão
cm = confusion_matrix(y_test, y_pred_fuzzy)

# Plotar
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(cm, cmap='Blues')

# Configurar eixos
ax.set_xticks(np.arange(len(class_names)))
ax.set_yticks(np.arange(len(class_names)))
ax.set_xticklabels(class_names)
ax.set_yticklabels(class_names)

# Rotacionar labels
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

# Adicionar valores nas células
for i in range(len(class_names)):
    for j in range(len(class_names)):
        text = ax.text(j, i, cm[i, j],
                       ha="center", va="center", 
                       color="white" if cm[i, j] > cm.max() / 2 else "black",
                       fontsize=16, fontweight='bold')

ax.set_title('Matriz de Confusão - Wang-Mendel', fontsize=14, fontweight='bold', pad=20)
ax.set_ylabel('Classe Real', fontsize=12)
ax.set_xlabel('Classe Predita', fontsize=12)
fig.colorbar(im, ax=ax)
plt.tight_layout()
plt.show()

print('✅ Matriz de confusão gerada!')

## 🔬 Passo 7: Comparação com Métodos Clássicos

In [ ]:
print('📊 Comparando com métodos clássicos de ML\n')

# K-Nearest Neighbors
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
acc_knn = accuracy_score(y_test, y_pred_knn)

# Decision Tree
dt = DecisionTreeClassifier(max_depth=4, random_state=42)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
acc_dt = accuracy_score(y_test, y_pred_dt)

print('='*70)
print('📋 COMPARAÇÃO DE MÉTODOS')
print('='*70)
print(f'   Wang-Mendel:    {acc_fuzzy*100:.2f}%')
print(f'   KNN (k=3):      {acc_knn*100:.2f}%')
print(f'   Decision Tree:  {acc_dt*100:.2f}%')
print('='*70)

## 📊 Gráfico de Comparação

In [ ]:
# Gráfico de barras
metodos = ['Wang-Mendel', 'KNN (k=3)', 'Decision Tree']
acuracias = [acc_fuzzy * 100, acc_knn * 100, acc_dt * 100]
cores = ['#FF6B6B', '#4ECDC4', '#45B7D1']

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(metodos, acuracias, color=cores, alpha=0.8, edgecolor='black', linewidth=1.5)

# Adicionar valores nas barras
for bar, acc in zip(bars, acuracias):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 1,
            f'{acc:.1f}%',
            ha='center', va='bottom', fontsize=13, fontweight='bold')

ax.set_ylabel('Acurácia (%)', fontsize=12)
ax.set_title('Comparação de Métodos de Classificação - Iris Dataset', 
             fontsize=14, fontweight='bold', pad=20)
ax.set_ylim([0, 110])
ax.grid(axis='y', alpha=0.3, linestyle='--')
plt.tight_layout()
plt.show()

print('✅ Comparação concluída!')
print('\n💡 Wang-Mendel tem acurácia competitiva COM a vantagem de ser interpretável!')

## 🔍 Passo 8: Exemplo de Predição Individual

Vamos ver como o sistema classifica uma amostra específica!

In [ ]:
# Escolher uma amostra de teste
idx_exemplo = 5
x_exemplo = X_test[idx_exemplo]
y_real = class_names[y_test[idx_exemplo]]

print('='*70)
print('🔍 EXEMPLO DE PREDIÇÃO INDIVIDUAL')
print('='*70)
print(f'\n📋 Amostra escolhida:')
print(f'   Petal Length: {x_exemplo[0]:.2f} cm')
print(f'   Petal Width:  {x_exemplo[1]:.2f} cm')
print(f'   Classe real:  {y_real}')
print()

# Fuzzificação
graus_length = wm.fis.inputs['petal_length'].fuzzify(x_exemplo[0])
graus_width = wm.fis.inputs['petal_width'].fuzzify(x_exemplo[1])

print('[ETAPA 1] FUZZIFICAÇÃO')
print('-'*70)
print(f'Petal Length = {x_exemplo[0]:.2f} cm:')
for termo, mu in graus_length.items():
    if mu > 0:
        print(f'  μ({termo}) = {mu:.3f}')

print(f'\nPetal Width = {x_exemplo[1]:.2f} cm:')
for termo, mu in graus_width.items():
    if mu > 0:
        print(f'  μ({termo}) = {mu:.3f}')

# Fazer predição
classe_pred_idx = wm.predict([x_exemplo])[0]
classe_pred = class_names[classe_pred_idx]

print()
print('='*70)
print(f'✅ Classe predita: {classe_pred}')
print(f'✅ Predição {"CORRETA" if classe_pred_idx == y_test[idx_exemplo] else "INCORRETA"}!')
print('='*70)

---

## 🎓 Conclusões

### ✅ Vantagens do Wang-Mendel para Classificação

1. **Interpretabilidade Total**
   - Regras linguísticas claras
   - Fácil entender COMO o sistema decide
   - Diferente de redes neurais (caixa-preta)

2. **Aprendizado Automático**
   - Regras geradas automaticamente dos dados
   - Não precisa de especialista
   - Rápido (um único passo pelos dados)

3. **Acurácia Competitiva**
   - Desempenho comparável a KNN e Decision Tree
   - Especialmente bom quando classes são linearmente separáveis

4. **Robustez**
   - Funciona bem mesmo com partições simples
   - Lida bem com dados fuzzy (fronteiras não nítidas)

---

### ⚠️ Limitações

1. **Particionamento Fixo**
   - Número de partições escolhido a priori
   - Pode não ser ótimo para todos os problemas

2. **Explosão de Regras**
   - Com muitas entradas: n_partitions^n_inputs regras possíveis
   - Para este exemplo: 5² = 25 regras possíveis

3. **Classificação Multiclasse**
   - Precisa mapear saída contínua para classes discretas
   - Pode haver ambiguidade nas fronteiras

---

### 🚀 Próximos Passos

1. **Experimentar com mais features**
   - Usar as 4 features do Iris
   - Ver como performance muda

2. **Otimizar partições**
   - Testar n_partitions = 3, 5, 7, 9
   - Encontrar melhor trade-off entre complexidade e acurácia

3. **Combinar com ANFIS**
   - Usar Wang-Mendel para inicializar ANFIS
   - Refinar regras com backpropagation (Aula 4!)

4. **Aplicar a outros datasets**
   - Wine Quality
   - Breast Cancer
   - Outros problemas de classificação

---

### 📚 Recursos

- **PyPI**: https://pypi.org/project/pyfuzzy-toolbox/
- **GitHub**: https://github.com/1moi6/pyfuzzy-toolbox

### 📖 Referências

1. **Wang, L. X., & Mendel, J. M. (1992)**. "Generating fuzzy rules by learning from examples." *IEEE Transactions on Systems, Man, and Cybernetics*, 22(6), 1414-1427.

2. **Fisher, R. A. (1936)**. "The use of multiple measurements in taxonomic problems." *Annals of Eugenics*, 7(2), 179-188.

3. **Ross, T. J. (2010)**. *Fuzzy Logic with Engineering Applications*. 3rd ed. Wiley.

---

**🎉 Parabéns! Você aplicou Wang-Mendel para classificação!**

*Notebook desenvolvido para o Minicurso de Sistemas de Inferência Fuzzy - Aula 3 - 2025*  
*Usando pyfuzzy-toolbox v1.0.0*